# Corpus paralelo Náhuatl (ncx) – Español (es) desde JW.org

**Autor:** preparado para Samuel Pérez Zistecatl  
**Objetivo:** Extraer y alinear verso por verso capítulos bíblicos publicados en JW.org en Náhuatl (del centro, `ncx`) y Español (`es`), produciendo:
- `corpus_ncx_es.csv` (tabla con columnas: `lang, libro, capitulo, versiculo, texto`)
- `parallel_ncx_es.jsonl` (líneas con `{src, tgt, libro, capitulo, versiculo}`)
- `metadata.json` (resumen, avisos de desalineación, etc.)

> ⚠️ **Uso responsable**: Revisa los **Términos de uso** y `robots.txt` de JW.org. Ejecuta con pausas (`sleep`) y solo con fines académicos.


## 0) Dependencias
Ejecuta esta celda **una vez** para instalar dependencias en tu entorno local (si hace falta). En algunos entornos puede ser `!pip` en vez de `%pip`.

```bash
%pip install requests beautifulsoup4 tqdm
```


In [1]:
# 1) Importaciones y constantes
import os
import re
import csv
import json
import time
import random
from urllib.parse import urljoin, urlparse

import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

BASE = "https://www.jw.org"

# Prefijos observados públicamente
LANG_PREFIX = {
    "es":  "es/biblioteca/biblia/nwt/libros",
    "ncx": "ncx/amatlajkuilolmej/biblia/nwt/libros",
}

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Corpus académico; contacto@example.com)",
    "Accept-Language": "es-ES,es;q=0.9",
}

class FetchError(Exception):
    pass


In [2]:
# 2) Utilidades de red y descubrimiento
def get(url: str, session: requests.Session, timeout: int = 20) -> requests.Response:
    """GET con control de errores y reintentos simples."""
    for intento in range(3):
        try:
            resp = session.get(url, headers=HEADERS, timeout=timeout)
            if resp.status_code == 200:
                return resp
            if resp.status_code == 404:
                return resp
            time.sleep(1.0 + intento)
        except requests.exceptions.RequestException:
            time.sleep(1.0 + intento)
    raise FetchError(f"No se pudo obtener {url} tras reintentos.")

def libros_disponibles(lang: str, session: requests.Session) -> list:
    """
    Descubre slugs de libros desde la página /libros/ del idioma dado.
    Devuelve una lista de slugs (p.ej., 'mateo', 'marcos', 'lucas', ...).
    """
    pref = LANG_PREFIX[lang].rstrip("/")
    url = f"{BASE}/{pref}/"
    resp = get(url, session)
    if resp.status_code != 200:
        raise FetchError(f"No se pudo acceder a {url} [{resp.status_code}]")
    soup = BeautifulSoup(resp.text, "html.parser")

    slugs = set()
    for a in soup.find_all("a", href=True):
        href = a["href"]
        if href.startswith("/"):
            full = urljoin(BASE, href)
        elif href.startswith("http"):
            full = href
        else:
            full = urljoin(url, href)

        parsed = urlparse(full)
        parts = [p for p in parsed.path.split("/") if p]
        try:
            idx = parts.index("libros")
        except ValueError:
            continue
        if idx + 1 < len(parts):
            slug = parts[idx + 1]
            if slug.isdigit():
                continue
            slugs.add(slug.strip().lower())

    return sorted(slugs)

def contar_capitulos(lang: str, libro: str, session: requests.Session, max_busqueda: int = 200) -> int:
    """Cuenta capítulos probando /<libro>/<n>/ hasta 404."""
    pref = LANG_PREFIX[lang].rstrip("/")
    for cap in range(1, max_busqueda + 1):
        url = f"{BASE}/{pref}/{libro}/{cap}/"
        resp = get(url, session)
        if resp.status_code == 404:
            return cap - 1
    return max_busqueda


In [3]:
# 3) Limpieza y extracción de versos
def limpiar_texto(texto: str) -> str:
    """Normaliza espacios y remueve símbolos aislados, conserva dígitos útiles."""
    texto = texto.replace("\u202f", " ").replace("\xa0", " ")
    texto = re.sub(r"\s*\*\s*", " ", texto)
    texto = re.sub(r"\s*\+\s*", " ", texto)
    texto = re.sub(r"[ \t]+", " ", texto)
    texto = re.sub(r"\s*\n\s*", "\n", texto)
    return texto.strip()

def extraer_versos_de_capitulo(html: str) -> list:
    """
    Intenta extraer un listado de versos en orden desde el contenedor principal.
    Heurística: buscar id='bibleText' y spans/p con id tipo v1,v2,... o data-pid secuencial.
    """
    soup = BeautifulSoup(html, "html.parser")
    root = soup.find(id="bibleText") or soup

    versos = []
    candidatos = []
    for tag in root.find_all(["span", "p"]):
        tid = (tag.get("id") or "").lower()
        dp = (tag.get("data-pid") or "").strip()
        if re.match(r"^v\d+$", tid) or dp.isdigit():
            candidatos.append(tag)

    if not candidatos:
        for p in root.find_all("p"):
            txt = p.get_text(" ", strip=True)
            if txt:
                partes = re.split(r"(?:(?<=\s)|^)\d{1,3}\s+", txt)
                for t in partes:
                    t = t.strip()
                    if t:
                        versos.append(t)
    else:
        for tag in candidatos:
            for sub in tag.find_all(["sup", "a", "span"], recursive=True):
                sub_text = (sub.get_text("", strip=True) or "").strip()
                if sub.name in {"sup"} or re.match(r"^\d{1,3}$", sub_text):
                    sub.decompose()
            t = tag.get_text(" ", strip=True)
            t = limpiar_texto(t)
            if t:
                versos.append(t)

    versos = [limpiar_texto(v) for v in versos if v and v.strip()]
    versos = [v for v in versos if not re.fullmatch(r"\d{1,3}", v)]
    return versos


In [4]:
# 4) Construcción del corpus (bucle principal)
def construir_corpus(outdir: str, sleep: float = 1.0, max_libros: int = 0) -> dict:
    os.makedirs(outdir, exist_ok=True)
    csv_path = os.path.join(outdir, "corpus_ncx_es.csv")
    jsonl_path = os.path.join(outdir, "parallel_ncx_es.jsonl")
    meta_path = os.path.join(outdir, "metadata.json")

    s = requests.Session()

    print("Descubriendo libros disponibles...")
    libros_ncx = libros_disponibles("ncx", s)
    libros_es = libros_disponibles("es", s)
    inter = [x for x in libros_ncx if x in set(libros_es)]
    if max_libros and max_libros > 0:
        inter = inter[:max_libros]

    meta = {
        "total_pairs": 0,
        "libros_interseccion": inter,
        "avisos": [],
        "por_libro": {},
    }

    with open(csv_path, "w", newline="", encoding="utf-8") as fcsv, open(jsonl_path, "w", encoding="utf-8") as fjsonl:
        w = csv.writer(fcsv)
        w.writerow(["lang", "libro", "capitulo", "versiculo", "texto"])

        for libro in inter:
            caps_ncx = contar_capitulos("ncx", libro, s)
            caps_es  = contar_capitulos("es",  libro, s)
            caps = min(caps_ncx, caps_es)
            meta["por_libro"][libro] = {"caps_ncx": caps_ncx, "caps_es": caps_es, "caps_usados": caps}

            for cap in tqdm(range(1, caps + 1), desc=f"{libro}", unit="cap"):
                url_ncx = f"{BASE}/{LANG_PREFIX['ncx']}/{libro}/{cap}/"
                url_es  = f"{BASE}/{LANG_PREFIX['es']}/{libro}/{cap}/"

                resp_ncx = get(url_ncx, s)
                resp_es  = get(url_es, s)

                versos_ncx = extraer_versos_de_capitulo(resp_ncx.text)
                versos_es  = extraer_versos_de_capitulo(resp_es.text)

                n, m = len(versos_ncx), len(versos_es)
                if n == 0 or m == 0:
                    meta["avisos"].append(f"Sin versos en {libro} {cap} (ncx={n}, es={m})")
                    continue

                L = min(n, m)
                if n != m:
                    meta["avisos"].append(f"Desalineación {libro} {cap}: ncx={n}, es={m}. Truncado a {L}.")

                for i in range(L):
                    v_idx = i + 1
                    src = versos_ncx[i]
                    tgt = versos_es[i]

                    w.writerow(["ncx", libro, cap, v_idx, src])
                    w.writerow(["es",  libro, cap, v_idx, tgt])

                    fjsonl.write(json.dumps({
                        "src": src,
                        "tgt": tgt,
                        "libro": libro,
                        "capitulo": cap,
                        "versiculo": v_idx,
                        "src_lang": "ncx",
                        "tgt_lang": "es",
                    }, ensure_ascii=False) + "\n")

                    meta["total_pairs"] += 1

                time.sleep(sleep + random.uniform(0, 0.5))

    with open(meta_path, "w", encoding="utf-8") as f:
        json.dump(meta, f, ensure_ascii=False, indent=2)

    print(f"\nListo. Guardado en:\n- {csv_path}\n- {jsonl_path}\n- {meta_path}")
    print(f"Total de pares alineados: {meta['total_pairs']}")
    if meta["avisos"]:
        print("\nAvisos (primeros 10):")
        for a in meta["avisos"][:10]:
            print(" -", a)
        if len(meta["avisos"]) > 10:
            print(f"... y {len(meta['avisos'])-10} avisos más.")

    return meta


In [6]:
# 5) Parámetros y ejecución
OUTDIR = r"C:\Users\Samuel Perez\Desktop\articulo"
       # Cambia la carpeta si lo prefieres
SLEEP = 1.2                 # Segundos de espera entre capítulos (respeto al servidor)
MAX_LIBROS = 0              # 0 = todos en intersección; p.ej., usa 2 para prueba rápida

# Ejecuta para construir el corpus
if __name__ == "__main__":
    meta = construir_corpus(outdir=OUTDIR, sleep=SLEEP, max_libros=MAX_LIBROS)
    # meta


Descubriendo libros disponibles...


deuteronomio:  85%|████████▌ | 29/34 [19:05<03:17, 39.51s/cap]


FetchError: No se pudo obtener https://www.jw.org/ncx/amatlajkuilolmej/biblia/nwt/libros/deuteronomio/30/ tras reintentos.

## 6) Siguientes pasos (opcional)
- Dividir `parallel_ncx_es.jsonl` en *train/valid/test*.
- Normalizar ortografía/tokenización para `ncx` según tus criterios.
- Exportar a formato plano (`src.txt`/`tgt.txt`) para tu pipeline de NMT.
- Añadir verificación automática de desalineaciones (por ejemplo, heurísticas por longitud).
